In [6]:
%%writefile explanation_engine.py

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

class ExplanationEngine:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float32
        )

        self.SYSTEM_PROMPT = """
You are a supply chain explanation assistant.

Rules:
- Explain using provided facts only
- Do not calculate or invent values
- Follow the format exactly

Format:

Cause:
<text>

Recommendation:
<text>

Justification:
<text>

Priority:
<High | Medium | Low>
"""

    def generate(self, facts: dict) -> str:
        prompt = f"""{self.SYSTEM_PROMPT}

FACTS:
{facts}

Answer:
"""

        inputs = self.tokenizer(prompt, return_tensors="pt")

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=120,
                do_sample=True,
                temperature=0.5,
                top_p=0.85,
                repetition_penalty=1.2,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.eos_token_id
            )

        gen_tokens = output[0][inputs["input_ids"].shape[-1]:]
        return self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

Overwriting explanation_engine.py


In [8]:
from explanation_engine import ExplanationEngine

engine = ExplanationEngine()

facts = {
    "product_name": "Product B",
    "stock_gap": 10,
    "demand_change_pct": 100,
    "lead_time_days": 7,
    "risk_days": 3.5,
    "priority": "High"
}

result = engine.generate(facts)
print(result)


The recommended action is to increase the lead time by 2 days due to a significant demand change of +10%. The product's stock gap has increased from 10 units to 12 units, which will require an additional 4 units to meet the demand. This leads to a total stock shortage of 6 units at the end of week 9.


In [9]:
from explanation_engine import ExplanationEngine

engine = ExplanationEngine()

facts = {
    "product_name": "Pen",
    "stock_gap": 10,
    "demand_change_pct": 50,
    "lead_time_days": 47,
    "risk_days": 2,
    "priority": "High"
}

result = engine.generate(facts)
print(result)

The demand for pens is expected to increase by 50% in the next month due to an unexpected product launch that will result in higher sales and stock levels. The lead time of 47 days has been reduced as the manufacturer has increased production capacity to meet the increased demand. This reduction in lead time reduces the risk associated with shortages during this period.
